# Generate composite galaxy

Finall, in this example we're going to demonstrate how to make a composite galaxy, including with imaging. For more information on defining parametric morphology see the Imaging examples.

In [ ]:
from unyt import yr, Myr, kpc, mas, Msun, Angstrom, deg, K
import matplotlib.pyplot as plt
import numpy as np

from synthesizer.grid import Grid
from synthesizer.dust.attenuation import PowerLaw
from synthesizer.dust.emission import Greybody
from synthesizer.parametric.morphology import Sersic2D, PointSource
from synthesizer.parametric import SFH, ZDist, Stars
from synthesizer.parametric.galaxy import Galaxy
from synthesizer.blackhole_emission_models import UnifiedAGN
from synthesizer.parametric import BlackHole
from synthesizer.sed import plot_spectra
from synthesizer.filters import UVJ

Let's begin by defining the geometry of the images:

In [ ]:
# Define geometry of the images
resolution = 0.01 * kpc  # resolution in kpc
npix = 301
fov = resolution * npix

Define the `grid_dir`:

In [ ]:
grid_dir = "../../../tests/test_grid"

### Diffuse ISM dust

Later on we're going to apply "diffuse" (ISM) dust attenuation to both the stars and blackholes so let's define these now. Note, these could be different for each component.

In [ ]:
tau_v = 0.5
# dust curve slope
alpha = -1.0
dust_curve = PowerLaw(slope=alpha)
dust_emission_model = Greybody(30 * K, 1.2)

## The Stars

Let's first build the stellar component of our galaxy, including setting the morphology so we can make an image later.

First define the grid. We need this to create our star formation and metal enrichment history.

In [ ]:
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

Let's define our star formation and metal enrichment history:

In [ ]:
stellar_mass = 10**14
sfh = SFH.Constant(duration=100 * Myr)
metal_dist = ZDist.Normal(mean=0.01, sigma=0.05)

Next, let's define the morphology:

In [ ]:
morph = Sersic2D(
    r_eff=1.0 * kpc, sersic_index=1.0, ellipticity=0.5, theta=35.0
)

Now let's initialise our stars object:

In [ ]:
# Get the 2D star formation and metal enrichment history for the given SPS grid. This is (age, Z).
stars = Stars(
    grid.log10age,
    grid.metallicity,
    sf_hist=sfh,
    metal_dist=metal_dist,
    initial_mass=stellar_mass,
    morphology=morph,
)

Now generate the stellar spectra:

In [ ]:
stars.get_spectra_pacman(
    grid, tau_v=tau_v, alpha=alpha, dust_emission_model=dust_emission_model
)

## The blackholes

Let's define the properties of our blackholes. We don't need to define the morphology since blackholes automatically assume a `PointSource` geometry.

In [ ]:
black_hole = BlackHole(
    mass=10**6.5 * Msun,
    inclination=60 * deg,
    accretion_rate=0.5 * Msun / yr,
    metallicity=0.01,
)

Next, let's define the emission model and generate the spectra.

In [ ]:
emission_model = UnifiedAGN(
    disc_model="test_grid_agn", photoionisation_model="", grid_dir=grid_dir
)

# Get the spectra assuming this emission model
spectra = black_hole.get_spectra_attenuated(
    emission_model,
    tau_v=tau_v,
    dust_curve=dust_curve,
    dust_emission_model=dust_emission_model,
)

## The Galaxy

Now that we have our components we can pass them to create an instance of a `Galaxy`.

In [ ]:
# Initialise Galaxy object
galaxy = Galaxy(stars=stars, black_holes=black_hole)
print(galaxy)

Now we have a galaxy with spectra isolated to the components. To get the combined "total", "emergent", and "intrinsic" spectra from all components we can call the `galaxy.get_spectra_combined`.

In [ ]:
galaxy.get_spectra_combined()

### Spectra

We can use the `plot_spectra` function to make a quick spectrum of the galaxy. 

In [ ]:
fig, ax = galaxy.plot_spectra(
    combined_spectra=True,
    stellar_spectra=True,
    black_hole_spectra=True,
    quantity_to_plot="luminosity",
    figsize=(10, 6),
)

### Images

To make iamges we need filters to define the photometry. For details see the [filter](../filters.ipynb) and [imaging](../imaging/imaging.rst) docs.

In [ ]:
filters = UVJ(new_lam=grid.lam)

With the spectra and filters made we can make images of each component and combine them.

In [ ]:
# Make images
img = galaxy.make_images(
    resolution=resolution,
    filters=filters,
    stellar_spectra_type="incident",
    blackhole_spectra_type="total",
    fov=fov,
)

# Make and plot an rgb image
img.make_rgb_image(rgb_filters={"R": "J", "G": "V", "B": "U"})
fig, ax, _ = img.plot_rgb_image(show=True)